## Prompt Engineering
- Try different prompts and formatting
- Check Results against human benchmark

In [1]:
import os
from openai import OpenAI
import json

## Load the Open AI API Key

In [2]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

api_key = os.getenv('OPENAI_API_KEY')

# Check if the API key is loaded
if api_key:
    print("API key is loaded.")
else:
    print("API key is not loaded.")

API key is loaded.


## Load a single test

In [6]:
with open('./Data/test.json', 'r') as file:
    test_json = json.load(file)
test_json

'{"Name":"The Fresh Market","Description":"Upscale grocery store chain with an old-world vibe offers local produce, prepared eats, wine & more.","Website":"https:\\/\\/stores.thefreshmarket.com\\/fl\\/orlando\\/38","Address":"5000 Dr Phillips Blvd, Orlando, FL 32819","Phone":"(407) 294-1516","Rating":4.5,"Price":"$$$","Type":["Grocery store","American grocery store","Bakery","Butcher shop","Fruit and vegetable store","Gourmet grocery store","Market","Organic food store","Produce market","Supermarket"],"Highlights":"Great produce","Offerings":"Organic products","Q_and_A":[{"question":{"user":{"name":"Anita Caragan","link":"https:\\/\\/www.google.com\\/maps\\/contrib\\/113357052055774270348","thumbnail":"https:\\/\\/lh3.googleusercontent.com\\/a\\/ACg8ocKdK8_h2Q72RjHiQIaBRzydgHNYXF_L4fVzUnKZqkzr=s120-c-rp-mo-br100"},"text":"Do you have bake chicken?","date":"4 years ago","language":"en"},"answer":{"user":{"name":"Jeanne M","link":"https:\\/\\/www.google.com\\/maps\\/contrib\\/10327323170

## Model 1: Minimal w/ Assistant API
- This prompt relies on the model's NLP ability to infer a match given our company description and the store information
- Store data includes reviews, snippets, short description, address
- API endpoint is the Assistants API, not the Chat Completions

In [7]:
client = OpenAI()

instructions = ("You are a helpful assistant tasked with helping us find grocery stores to stock our honey. "
                "We are a fourth-generation, family-owned apiary based out of northern Florida that specializes "
                "in the production of extremely high quality, mono-floral honey. We are looking for grocers that "
                "specialize in curating gourmet food products, especially from small businesses such as ours. "
                "Your task is to assign a compatibility score from 1-10 to the grocery store based on the {json} "
                "of store data we provide. The output should be a json with the score and a short description "
                "justifying why a given score was received.")

assistant = client.beta.assistants.create(
    name="Gourmet Grocery Store Finder (Minimal)",
    instructions=instructions,
    model="gpt-4-1106-preview"
)

#### Generate a Run

In [8]:
thread = client.beta.threads.create()

In [9]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=test_json
)

In [32]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [38]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
messages

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_VXClhv38cBJWwWUuH3xMVdQu', assistant_id='asst_d1LgL4keVHGdV8XmKyetMtDO', content=[MessageContentText(text=Text(annotations=[], value='```json\n{\n  "Score": 9,\n  "Description": "The Fresh Market receives a compatibility score of 9 as it aligns closely with the criterion for a grocery store that specializes in curating gourmet food products. Given its upscale and gourmet focus, it should be a natural fit for high quality, mono-floral honey from a family-owned small business. The store\'s 4.5 rating and range of offerings, including organic products and a reputation for great produce, enhance the likelihood of a mutually beneficial partnership. Additionally, customer reviews indicate a preference for unique and high-quality items, which further supports the compatibility."\n}\n```'), type='text')], created_at=1705963418, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_t43dmvG5We0tfjwYFWcT1vzJ', th

#### Figure out the output format

In [39]:
messages.data[i].content[0].text.value

'```json\n{\n  "Score": 9,\n  "Description": "The Fresh Market receives a compatibility score of 9 as it aligns closely with the criterion for a grocery store that specializes in curating gourmet food products. Given its upscale and gourmet focus, it should be a natural fit for high quality, mono-floral honey from a family-owned small business. The store\'s 4.5 rating and range of offerings, including organic products and a reputation for great produce, enhance the likelihood of a mutually beneficial partnership. Additionally, customer reviews indicate a preference for unique and high-quality items, which further supports the compatibility."\n}\n```'

In [40]:
for i in range(len(messages.data)):
    print(i,messages.data[i].role)
    print("\t",messages.data[i].content[0].text.value)


0 assistant
	 ```json
{
  "Score": 9,
  "Description": "The Fresh Market receives a compatibility score of 9 as it aligns closely with the criterion for a grocery store that specializes in curating gourmet food products. Given its upscale and gourmet focus, it should be a natural fit for high quality, mono-floral honey from a family-owned small business. The store's 4.5 rating and range of offerings, including organic products and a reputation for great produce, enhance the likelihood of a mutually beneficial partnership. Additionally, customer reviews indicate a preference for unique and high-quality items, which further supports the compatibility."
}
```
1 user
	 {"Name":"The Fresh Market","Description":"Upscale grocery store chain with an old-world vibe offers local produce, prepared eats, wine & more.","Website":"https:\/\/stores.thefreshmarket.com\/fl\/orlando\/38","Address":"5000 Dr Phillips Blvd, Orlando, FL 32819","Phone":"(407) 294-1516","Rating":4.5,"Price":"$$$","Type":["Gro

#### Clean and evaluate

In [41]:
json_response = messages.data[0].content[0].text.value
cleaned_json_string = json_response.strip('```json\n').rstrip('\n```')
# Parse the JSON string
data = json.loads(cleaned_json_string)

# Accessing data
score = data.get('Score')
description = data.get('Description')

print("Score:", score)
print("Description:", description)

Score: 9
Description: The Fresh Market receives a compatibility score of 9 as it aligns closely with the criterion for a grocery store that specializes in curating gourmet food products. Given its upscale and gourmet focus, it should be a natural fit for high quality, mono-floral honey from a family-owned small business. The store's 4.5 rating and range of offerings, including organic products and a reputation for great produce, enhance the likelihood of a mutually beneficial partnership. Additionally, customer reviews indicate a preference for unique and high-quality items, which further supports the compatibility.


## That works, but I should probably switch to chat completions since the assistants API costs additional money and I don't need any of the features